In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show selenium

Name: selenium
Version: 4.38.0
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache-2.0
Location: C:\Users\wauin\anaconda3\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install selenium google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ------ --------------------------------- 2.4/14.6 MB 13.4 MB/s eta 0:00:01
   ------------ --------------------------- 4.5/14.6 MB 11.7 MB/s eta 0:00:01
   ------------------- -------------------- 7.1/14.6 MB 11.8 MB/s eta 0:00:01
   --------------------------- ------------ 10.0/14.6 MB 12.2 MB/s eta 0:00:01
   ----------------------------------- ---- 13.1/14.6 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 12.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
#重複防止
import datetime
import os.path
import time
import json
import hashlib  # 【追加】ID生成用

# --- Google Calendar API関連のインポート ---
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.exceptions import RefreshError
from googleapiclient.errors import HttpError # 【追加】APIエラーハンドリング用

# --- Selenium関連のインポート ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==========================================
# 1. Google Calendar API 設定エリア
# ==========================================
SCOPES = ['https://www.googleapis.com/auth/calendar']
CALENDAR_ID = 'primary'

def get_calendar_service():
    """認証を行ってAPIサービスを返す関数"""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except RefreshError:
                print("権限が変更されたため、再認証を行います。")
                creds = None
        
        if not creds:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return build('calendar', 'v3', credentials=creds)

def parse_deadline_str(date_str):
    """日付文字列をdatetimeオブジェクトに変換する補助関数"""
    if "～" in date_str:
        date_str = date_str.split("～")[-1].strip()
    
    now = datetime.datetime.now()
    current_year = now.year

    try:
        dt = datetime.datetime.strptime(f"{current_year}/{date_str}", '%Y/%m/%d %H:%M')
        # 年またぎ対応
        if now.month == 12 and dt.month == 1:
            dt = dt.replace(year=current_year + 1)
        return dt
    except ValueError:
        print(f"⚠️ 日付変換エラー: {date_str} -> 明日の日付で登録します")
        return now + datetime.timedelta(days=1)

def generate_event_id(subject, title, deadline_str):
    """
    科目名、課題名、期限の文字列から一意のID（ハッシュ値）を生成する。
    同じ課題なら常に同じIDになるため、重複判定に利用できる。
    GoogleカレンダーのIDに使用できる文字(0-9, a-v)に合わせるため、
    MD5ハッシュ(0-9, a-f)を使用する。
    """
    raw_string = f"{subject}_{title}_{deadline_str}"
    # 文字列をバイト列にしてMD5ハッシュ化し、16進数文字列で返す
    return hashlib.md5(raw_string.encode('utf-8')).hexdigest()

def register_assignment(service, subject, title, deadline_str):
    """
    課題をカレンダーに登録する関数（重複チェック付き）
    """
    # 日時変換
    deadline_dt = parse_deadline_str(deadline_str)
    start_dt = deadline_dt - datetime.timedelta(hours=1)

    # 重複防止用の一意なIDを生成
    event_id = generate_event_id(subject, title, deadline_str)

    # Googleカレンダー用のデータを作成
    event_body = {
        'id': event_id,  # 【重要】ここでIDを指定する
        'summary': f"【{subject}】{title}",
        'description': f'ポータルサイトから自動連携\n元の期限表記: {deadline_str}',
        'start': {
            'dateTime': start_dt.isoformat(),
            'timeZone': 'Asia/Tokyo',
        },
        'end': {
            'dateTime': deadline_dt.isoformat(),
            'timeZone': 'Asia/Tokyo',
        },
        'colorId': '11', # 赤色
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'popup', 'minutes': 60 * 24},
                {'method': 'popup', 'minutes': 60},
            ],
        },
    }

    # カレンダーに登録
    try:
        event = service.events().insert(calendarId='primary', body=event_body).execute()
        print(f"📅 新規登録成功！: {title} (期限: {deadline_str})")
    
    except HttpError as error:
        # ステータスコードが409の場合は「既に存在する」という意味
        if error.resp.status == 409:
            print(f"🔄 既に登録済みのためスキップ: {title}")
        else:
            # それ以外のエラーは表示する
            print(f"❌ 登録エラー: {error}")
    except Exception as e:
        print(f"❌ 予期せぬエラー: {e}")


# ==========================================
# 2. メイン実行ブロック
# ==========================================
if __name__ == '__main__':
    print("--- 自動課題登録システム起動 ---")

    # Chrome起動
    driver = webdriver.Chrome()

    all_assignments_data = []

    try:
        # PLASログインページへアクセス
        driver.get("https://plas.soka.ac.jp/csp/plassm/index.csp")
        time.sleep(3)
        
        target_style_value ="margin-top:40px;text-align:center;font-size:18px;"
        css_selector = f'div[style="{target_style_value}"]'
        
        try:
            wait = WebDriverWait(driver, 10)
            element_to_click = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
            )
            element_to_click.click()
        except Exception as e:
            print(f"❌ 要素が見つからなかったか、クリックできませんでした。エラー: {e}")

        # ユーザー入力
        print("ユーザーIDを入力してください")
        username_input = (input())
        
        try:
            wait = WebDriverWait(driver, 10)
            username_field = wait.until(
                EC.presence_of_element_located((By.ID, "plas-username"))
            )
            username_field.clear() 
            username_field.send_keys(username_input)
            print(f"✅ ユーザーIDを入力しました。")
        except Exception as e:
            print(f"❌ ユーザー名フィールドエラー: {e}")

        print('パスワードを入力してください')
        password_input = (input())
        
        try:
            wait = WebDriverWait(driver, 10)
            password_field = wait.until(
                EC.presence_of_element_located((By.ID, "plas-password"))
            )
            password_field.clear() 
            password_field.send_keys(password_input)
            print(f"✅ パスワードを入力しました。")
        except Exception as e:
            print(f"❌ パスワードフィールドエラー: {e}")
        
        # ログイン
        try:
            wait = WebDriverWait(driver, 10)
            css_selector = 'button[type="submit"]' 
            login_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
            )
            login_button.click()
            print("✅ ログインボタンをクリックしました。")
        except Exception as e:
            print(f"❌ ログインボタンエラー: {e}")

        # 課題ページへ
        try:
            wait = WebDriverWait(driver, 10)
            css_selector = 'a.btn-func4' 
            assignment_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
            )
            assignment_button.click()
            print("✅ 「授業課題」ボタンをクリックしました。")
        except Exception as e:
            print(f"❌ 「授業課題」ボタンエラー: {e}")

        # データ抽出
        assignment_containers_selector = 'div.box-div.contents-box' 
        try:
            wait = WebDriverWait(driver, 15) 
            wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, assignment_containers_selector))
            )
            
            assignment_containers = driver.find_elements(By.CSS_SELECTOR, assignment_containers_selector)
            print(f"✅ 合計 {len(assignment_containers)} 個の科目コンテナが見つかりました。解析を開始します...")

            for container in assignment_containers:
                try:
                    kamokuname = container.find_element(By.CSS_SELECTOR, '.header-kamokuname').text.strip()
                except Exception:
                    kamokuname = "【科目名不明】"
                
                assignment_name_selector = '.kamoku-info.col-sm-5.text-contents.contents-flex'
                assignment_name_elements = container.find_elements(By.CSS_SELECTOR, assignment_name_selector)
                end_date_elements = container.find_elements(By.CSS_SELECTOR, 'span.em')

                for assign_el, date_el in zip(assignment_name_elements, end_date_elements):
                    try:
                        assignment_name = assign_el.text.strip()
                        end_date = date_el.text.strip()
                        
                        all_assignments_data.append({
                            'subject': kamokuname,
                            'title': assignment_name,
                            'deadline': end_date
                        })
                    except Exception as e:
                        print(f"⚠️ データ抽出中エラー: {e}")
        except Exception as e:
            print(f"❌ 課題データ取得中エラー: {e}")

    finally:
        driver.quit()

    # ==========================================
    # 3. カレンダーへの登録処理
    # ==========================================
    print("\n" + "="*50)
    print(f"抽出完了: {len(all_assignments_data)} 件の課題が見つかりました。")
    print("Googleカレンダーへの登録を開始します...")
    print("="*50)

    if all_assignments_data:
        service = get_calendar_service()
        for data in all_assignments_data:
            register_assignment(
                service,
                data['subject'],
                data['title'],
                data['deadline']
            )
    else:
        print("登録する課題がありませんでした。")
    
    print("--- 全ての処理が完了しました ---")

--- 自動課題登録システム起動 ---
ユーザーIDを入力してください


 e2458202


✅ ユーザーIDを入力しました。
パスワードを入力してください


 H17acba4386


✅ パスワードを入力しました。
✅ ログインボタンをクリックしました。
✅ 「授業課題」ボタンをクリックしました。
✅ 合計 5 個の科目コンテナが見つかりました。解析を開始します...

抽出完了: 6 件の課題が見つかりました。
Googleカレンダーへの登録を開始します...
🔄 既に登録済みのためスキップ: 第12回_授業リフレクションシート
🔄 既に登録済みのためスキップ: 最終プロジェクト一式
🔄 既に登録済みのためスキップ: プレゼンテーション
🔄 既に登録済みのためスキップ: 課題5
🔄 既に登録済みのためスキップ: スライド提出
🔄 既に登録済みのためスキップ: 自然科学と芸術について
--- 全ての処理が完了しました ---
